In [ ]:
!unzip 20_newsgroup.zip

In [5]:
import os
import sys
import joblib
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from keras.models import Model
from keras.initializers import Constant

BASE_DIR = ''

# glove模型路径
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
# 文本语料路径
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')

# 模型参数
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

# 读取词向量
print('Indexing word vectors.')
embeddings_index = {}
with open(os.path.join(BASE_DIR, 'glove.6B.100d.txt'), 'r', encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

# 获取训练文本与标签
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')  # skip header
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)
print('Get training text and labels: found %s texts.' % len(texts))

# 使用token分词把词转化为标号
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
joblib.dump(tokenizer, 'token_result.pkl')
print('Transforms words into labels by token.')

# 将词向量字典传入tokenizer并根据将训练数据转化为sequences
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
# print(data)
labels = to_categorical(np.asarray(labels))
# print(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
indices = np.arange(data.shape[0])

# 划分训练集和测试集
np.random.shuffle(indices)
data = data[indices]
# print(data)
labels = labels[indices]
# print(labels)
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]
print('Dividing training set and test set.')

# 准备嵌入矩阵
print('Preparing embedding matrix.')
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        # 从预训练模型的词向量到语料库的词向量映射
        embedding_matrix[i] = embedding_vector

# 将预先训练的单词放入嵌入层并保持嵌入固定
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
print('Training model.')

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

Indexing word vectors.
Get training text and labels: found 19997 texts.
Transforms words into labels by token.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)
Dividing training set and test set.
Preparing embedding matrix.
Training model.


In [5]:
import tensorflow as tf

x = Conv1D(128, 5, activation='relu')(embedded_sequences)    #输出128维，卷积窗口大小为5
tf.keras.regularizers.l2(0.1)(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)
preds = Dense(len(labels_index), activation='softmax')(x)

# 训练模型
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy', optimizer='Adagrad', metrics=['acc'])
CNN_model=model.fit(x_train, y_train,
                  batch_size=128,
                  epochs=20,
                  validation_data=(x_val, y_val))

Train on 15998 samples, validate on 3999 samples
Epoch 1/20
15998/15998 [==============================] - 4s 243us/step - loss: 2.4134 - acc: 0.2248 - val_loss: 1.6438 - val_acc: 0.4364
Epoch 2/20
15998/15998 [==============================] - 1s 80us/step - loss: 1.4153 - acc: 0.4914 - val_loss: 1.2057 - val_acc: 0.5724
Epoch 3/20
15998/15998 [==============================] - 1s 80us/step - loss: 1.0948 - acc: 0.6174 - val_loss: 1.0870 - val_acc: 0.6302
Epoch 4/20
15998/15998 [==============================] - 1s 80us/step - loss: 0.8833 - acc: 0.6938 - val_loss: 0.9448 - val_acc: 0.6842
Epoch 5/20
15998/15998 [==============================] - 1s 80us/step - loss: 0.7439 - acc: 0.7480 - val_loss: 0.9200 - val_acc: 0.7032
Epoch 6/20
15998/15998 [==============================] - 1s 80us/step - loss: 0.6458 - acc: 0.7805 - val_loss: 0.9216 - val_acc: 0.6969
Epoch 7/20
15998/15998 [==============================] - 1s 80us/step - loss: 0.5593 - acc: 0.8145 - val_loss: 0.8694 - val_acc

In [ ]:
# import keras

# max_features = 20000
# batch_size = 8192

# model = keras.models.Sequential()
# model.add(Embedding(max_features, 128))
# model.add(LSTM(128, dropout=0.2, return_sequences=False, recurrent_dropout=0.2))
# model.add(Dense(20, activation='softmax'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(x_train, y_train, batch_size=batch_size, epochs=2, validation_data=(x_val, y_val))
# print('Testing on x_val.')
# score, acc = model.evaluate(x_val, y_val, batch_size=batch_size)
# print('Accuracy on x_val: %.2f%%'%(acc*100))
# print('Binary_crossentropy on x_val: %.2f%%'%(score))

In [4]:
import keras
from keras.layers import CuDNNLSTM
# 模型搭建
model = keras.models.Sequential()
model.add(embedding_layer)#嵌入层
model.add(CuDNNLSTM(128))#LSTM层
model.add(Dense(128,activation='relu'))#全连接层
model.add(Dense(20, activation='softmax'))
model.compile(loss="categorical_crossentropy",
              optimizer='rmsprop', metrics=['accuracy'])

# 训练
LSTM_model=model.fit(x_train, y_train, batch_size=64,
            epochs=20, validation_data=(x_val, y_val))

Train on 15998 samples, validate on 3999 samples
Epoch 1/20
15998/15998 [==============================] - 14s 905us/step - loss: 2.5490 - acc: 0.1854 - val_loss: 2.1758 - val_acc: 0.2628
Epoch 2/20
15998/15998 [==============================] - 14s 887us/step - loss: 2.0205 - acc: 0.3287 - val_loss: 2.5957 - val_acc: 0.2351
Epoch 3/20
15998/15998 [==============================] - 14s 892us/step - loss: 1.6304 - acc: 0.4487 - val_loss: 1.4841 - val_acc: 0.4954
Epoch 4/20
15998/15998 [==============================] - 14s 893us/step - loss: 1.3497 - acc: 0.5421 - val_loss: 1.2601 - val_acc: 0.5679
Epoch 5/20
15998/15998 [==============================] - 14s 895us/step - loss: 1.1335 - acc: 0.6151 - val_loss: 1.1418 - val_acc: 0.6122
Epoch 6/20
15998/15998 [==============================] - 14s 895us/step - loss: 0.9855 - acc: 0.6681 - val_loss: 1.0660 - val_acc: 0.6399
Epoch 7/20
15998/15998 [==============================] - 14s 898us/step - loss: 0.8693 - acc: 0.7060 - val_loss: 0.9